In [1]:
'''
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""
'''

'\nimport os\nos.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152\nos.environ["CUDA_VISIBLE_DEVICES"] = ""\n'

In [2]:
import gym
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
#from keras.constraints import MaxNorm
import numpy as np
import tensorflow as tf
import gc
import pickle
from keras.models import load_model

Using TensorFlow backend.


In [3]:
#Checking use of GPUs
from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())
sess= K.get_session()

['/gpu:0']


In [4]:
#Initializing environment
env = gym.make('Humanoid-v2')
So = env.reset()
A = env.action_space.sample()

In [5]:
#Creating the model for policy
global policy_model
policy_model = Sequential()
policy_model.add(Dense(400,input_shape = np.append(So, A).shape,kernel_initializer='random_uniform', activation = 'sigmoid' ))
policy_model.add(Dropout(0.5))
policy_model.add(Dense(100,kernel_initializer='random_uniform', activation = 'sigmoid'))
policy_model.add(Dropout(0.5))
policy_model.add(Dense(30,kernel_initializer='random_uniform', activation = 'sigmoid'))
policy_model.add(Dense(1,kernel_initializer='random_uniform', activation= 'sigmoid'))

#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
policy_model.compile(loss="mse", optimizer= "sgd")
print(policy_model.summary())
policy_model.load_weights("policy.h5")

sess.run(tf.global_variables_initializer())

#Intializing eligibility traces for policy improvement
global policy_eligibility_traces
policy_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in policy_model.trainable_weights]

#Setting learning hyperparameters 
global policy_alpha
policy_alpha = 0.1
global policy_lambda
policy_lambda =  0.9

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 400)               157600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               40100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 31        
Total params: 200,761
Trainable params: 200,761
Non-trainable params: 0
_________________________________________________________________
None

In [6]:
#Creating model for values 
global value_model
value_model = Sequential()
value_model.add(Dense(400,input_shape = So.shape, kernel_initializer='random_uniform',activation = 'relu' ))
value_model.add(Dropout(0.5))
value_model.add(Dense(100,kernel_initializer='random_uniform', activation = 'relu'))
value_model.add(Dropout(0.5))
value_model.add(Dense(30, kernel_initializer='random_uniform',activation = 'relu'))
value_model.add(Dense(1,kernel_initializer='random_uniform', activation= 'linear'))

#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
value_model.compile(loss="mse", optimizer= "sgd")
print(value_model.summary())
value_model.load_weights("value.h5")

sess.run(tf.global_variables_initializer())

#Intializing eligibility traces for value improvement
global value_eligibility_traces
value_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in value_model.trainable_weights]

#Setting learning hyperparameters 
global value_alpha
value_alpha = 0.1
global value_lambda
value_lambda = 0.9

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 400)               150800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               40100     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 31        
Total params: 193,961
Trainable params: 193,961
Non-trainable params: 0
_________________________________________________________________
None

In [7]:
#Defining other hyperparameters

global average_reward
average_reward = pickle.load(open('average_reward.ferch','rb'))
global etaa
etaa = 0.01

In [8]:
#Auxiliary functions for weights update

def set_weights(model, weights):  
    if(model=='value'):
        i=0
        for layer in value_model.layers:
            if 'dropout' not in layer.name:
                layer.set_weights([weights[i],weights[i+1]])
                i+=2
    elif(model=='policy'):
        i=0
        for layerz in policy_model.layers:
            if 'dropout' not in layerz.name:
                layerz.set_weights([weights[i],weights[i+1]])
                i+=2
    else:
        print('error')

def get_value_weights():
    weights =[]
    for w in value_model.layers:
        weights+= w.weights
    return weights

def get_policy_weights():
    weights =[]
    for w in policy_model.layers:
        weights+= w.weights
    return weights

In [9]:
#Reward Tracking
rewardz_received = []
delta_rewardz_received = []

In [10]:
#Update weights
def update_weights(previous_state, action , reward, state,  terminal):
    
    #Compute rewards
    global average_reward,rewardz_received,delta_rewardz_received
    delta_rewards = np.exp(reward) - average_reward + value_model.predict(np.array([state]))[0][0] - value_model.predict(np.array([previous_state]))[0][0]    
    rewardz_received.append(reward)
    delta_rewardz_received.append(delta_rewards)
    if(terminal):
        delta_rewards = reward - average_reward + 0 - value_model.predict(np.array([previous_state]))[0][0]
    if(delta_rewards>=10):
        delta_rewards =10
    if(delta_rewards<=-10):
        delta_rewards=-10
    average_reward = average_reward + etaa*delta_rewards
    
    
    #Compute value updates (eligibility traces and weights)
    global value_eligibility_traces
    value_gradients = value_model.optimizer.get_gradients(value_model.output, value_model.trainable_weights)
    value_gradients = [tf.clip_by_norm(gradient, 10) for gradient in value_gradients]
    value_eligibility_traces_op = [tf.add(tf.multiply(value_eligibility_traces[i], tf.constant(value_lambda,dtype= tf.float32)),value_gradients[i] ) for i in range(len(value_eligibility_traces))] 
    value_eligibility_traces = [tf.convert_to_tensor(x) for x in sess.run(value_eligibility_traces_op,feed_dict={value_model.input:np.array([previous_state])})]       
    value_weights = get_value_weights()
    value_weights_op =  [tf.add(value_weights[i], tf.multiply(tf.constant(value_alpha*delta_rewards,dtype=tf.float32), value_eligibility_traces[i])) for i in range(len(value_weights))]
    set_weights('value', sess.run(value_weights_op))
    
    #Compute policy updates (eligibility traces and weights)
    global policy_eligibility_traces
    policy_gradients = policy_model.optimizer.get_gradients(policy_model.output, policy_model.trainable_weights)
    policy_gradients = [tf.clip_by_norm(gradient, 10) for gradient in policy_gradients]
    policy_value = policy_model.predict(np.array([np.append(previous_state, action)]))[0][0]
    policy_value = 1/ (policy_value+1)
    policy_eligibility_traces_op = [tf.add(tf.multiply(tf.constant(policy_lambda,dtype=tf.float32), policy_eligibility_traces[i]), tf.multiply(tf.constant(policy_value, dtype= tf.float32), policy_gradients[i])) for i in range(len(policy_gradients))]
    policy_eligibility_traces = [tf.convert_to_tensor(x) for x in sess.run(policy_eligibility_traces_op, feed_dict={policy_model.input:np.array([np.append(previous_state,action)])})]
    policy_weights = get_policy_weights()
    policy_weights_op =  [tf.add(policy_weights[i], tf.multiply( policy_eligibility_traces[i],tf.constant(policy_alpha*delta_rewards,dtype=tf.float32))) for i in range(len(policy_weights))]
    set_weights('policy', sess.run(policy_weights_op))

In [11]:
#Creating model for remembering preferences over each state
number_of_preferences =10
global preference_model
preference_model = Sequential()
preference_model.add(Dense(400,input_shape = So.shape, activation = 'relu' ))
preference_model.add(Dropout(0.5))
preference_model.add(Dense(100, activation = 'relu'))
preference_model.add(Dropout(0.5))
preference_model.add(Dense(30, activation = 'relu'))
preference_model.add(Dense(A.shape[0]*number_of_preferences, activation= 'linear'))

adam = Adam(lr=0.4, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
preference_model.compile(loss="mse", optimizer=adam)
print(preference_model.summary())
preference_model.load_weights('preference.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 400)               150800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)               40100     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 30)                3030      
_________________________________________________________________
dense_12 (Dense)             (None, 170)               5270      
Total params: 199,200
Trainable params: 199,200
Non-trainable params: 0
_________________________________________________________________
None

In [ ]:
#Selecting an action
def select_action(state):
    
    number_of_new_actions = 3
    
    predicted_preferences = preference_model.predict(np.array([state]))[0].reshape((number_of_preferences,A.shape[0]))
    
    #FIXING MATRIX INCONSISTENCIES
    low = env.action_space.low
    high = env.action_space.high
    
    for y in range(predicted_preferences.shape[0]):
        for z in range(predicted_preferences.shape[1]):
            if(predicted_preferences[y][z] < low[z]):
                predicted_preferences[y][z] = low[z]
            if(predicted_preferences[y][z] > high[z]):
                predicted_preferences[y][z] = high[z]
                
    new_actions = env.action_space.sample().reshape(1, A.shape[0])
    
    for _ in range(number_of_new_actions-1):
        new_actions= np.concatenate((new_actions,env.action_space.sample().reshape(1, A.shape[0])), axis=0 )
    
    all_actions = np.concatenate((predicted_preferences, new_actions), axis=0)
    all_actionz = np.concatenate((np.tile(So, (all_actions.shape[0], 1)), all_actions),axis=1)
    policy_preferences = policy_model.predict(all_actionz)
    #print(policy_preferences)
    
    actions_list = []
    for i in range(all_actions.shape[0]):
        actions_list.append((all_actions[i], policy_preferences[i][0]))
    actions_list = sorted(actions_list, key = lambda x: x[1], reverse = True)
    
    probabilities = sess.run(tf.contrib.layers.softmax(tf.constant([[x[1] for x in actions_list]])))[0]
    p = np.random.rand()
    i=0
    #print(probabilities)
    for j in range(probabilities.shape[0]):
        p-=probabilities[j]
        if(p<0):
            i=j
    
    best_actions = np.array([x[0] for x in actions_list[0:number_of_preferences]]).flatten()
    preference_model.fit(np.array([state]), np.array([best_actions]), verbose=0)
    return actions_list[j][0]
        
    

In [ ]:
#Main loop
S = env.reset()
#action_count=0
episode_count=0
save= True
total_reward =0 
total_reward_list=[]
while(True):
    
    #renders environment 
    env.render()
    
    
    #Selects action according to stochastic policy
    action = select_action(S)
    #action_count+=1
    
    #Takes action 
    S1, reward, done, info = env.step(action)
    
    
    #Updates weights
    #if(np.random.randint(10)%5==0):
    update_weights(S , action, reward, S1 , done)
    
    S = S1
    total_reward +=reward
    
    if(done):
        
        print('We are now at '+str(episode_count))
        gc.collect()
        print('total reward '+ str(total_reward))
        total_reward_list.append(total_reward)
        total_reward=0
        
        #Resets episode
        S = env.reset()
        #action_count=0
        episode_count+=1
        
        
        
        if(episode_count%5==0):
            
            if(save):
                print('saving models')
                print('average reward '+ str(average_reward))
                
                preference_model.save('preference.h5')
                policy_model.save('policy.h5')
                value_model.save('value.h5')
                
                pickle.dump(rewardz_received, open('rewardz_received.ferch','wb'))
                pickle.dump(delta_rewardz_received, open('delta_rewardz_received.ferch','wb'))
                pickle.dump(average_reward, open('average_reward.ferch','wb'))
                pickle.dump(total_reward_list, open('total_rewards_hist','wb'))
                
                #Restarting keras session 
                K.clear_session()
                sess = K.get_session()
                preference_model = load_model('preference.h5')
                policy_model = load_model('policy.h5')
                value_model = load_model('value.h5')

        
        #Resets eligibility traces
        global value_eligibility_traces, policy_eligibility_traces
        policy_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in policy_model.trainable_weights]
        value_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in value_model.trainable_weights]
    

Creating window glfw
We are now at 0
total reward 109.113840109
We are now at 1
total reward 242.31404461
We are now at 2
total reward 97.5434779521
We are now at 3
total reward 109.121320867
We are now at 4
total reward 112.92640442
saving models
average reward 135.011564136
We are now at 5
total reward 101.260694043
We are now at 6
total reward 140.302279725
We are now at 7
total reward 95.9761549012
We are now at 8
total reward 93.7178021922
We are now at 9
total reward 82.7045732193
saving models
average reward 133.90937266
We are now at 10
total reward 78.262124873
We are now at 11
total reward 110.171949576
We are now at 12
total reward 114.799557196
We are now at 13
total reward 184.885431268
We are now at 14
total reward 82.7750815417
saving models
average reward 132.799879099
We are now at 15
total reward 143.321806366
We are now at 16
total reward 100.672591852
We are now at 17
total reward 105.620675061
We are now at 18
total reward 116.473664581
We are now at 19
total rewar

We are now at 155
total reward 92.5343928927
We are now at 156
total reward 83.2632247078
We are now at 157
total reward 85.0355704533
We are now at 158
total reward 91.5159002778
We are now at 159
total reward 89.7506002589
saving models
average reward 115.254672309
We are now at 160
total reward 86.1272589926
We are now at 161
total reward 89.496773242
We are now at 162
total reward 88.8969438132
We are now at 163
total reward 85.9780533117
We are now at 164
total reward 84.6012223026
saving models
average reward 114.818287167
We are now at 165
total reward 83.0256401584
We are now at 166
total reward 89.8721878512
We are now at 167
total reward 86.5603282693
We are now at 168
total reward 88.8186866781
We are now at 169
total reward 91.7811519683
saving models
average reward 114.813385391
We are now at 170
total reward 84.3626279384
We are now at 171
total reward 81.7861255554
We are now at 172
total reward 83.8643402771
We are now at 173
total reward 84.60211608
We are now at 174
t

We are now at 309
total reward 75.8858267736
saving models
average reward 115.814182883
We are now at 310
total reward 76.2446209641
We are now at 311
total reward 69.4618817627
We are now at 312
total reward 77.7786546099
We are now at 313
total reward 76.0694897432
We are now at 314
total reward 79.1164403003
saving models
average reward 115.637716688
We are now at 315
total reward 76.2059914211
We are now at 316
total reward 88.1069637933
We are now at 317
total reward 82.9912371407
We are now at 318
total reward 71.9313250514
We are now at 319
total reward 76.0965174798
saving models
average reward 115.889197219
We are now at 320
total reward 76.1418838147
We are now at 321
total reward 83.0376803688
We are now at 322
total reward 85.4666884507
We are now at 323
total reward 82.9631009401
We are now at 324
total reward 79.1130534499
saving models
average reward 115.935490893
We are now at 325
total reward 71.2767885554
We are now at 326
total reward 76.0417644992
We are now at 327


We are now at 462
total reward 49.0491466998
We are now at 463
total reward 43.8328352883
We are now at 464
total reward 44.1441438693
saving models
average reward 109.73650909
We are now at 465
total reward 43.9488871448
We are now at 466
total reward 44.1378384484
We are now at 467
total reward 44.1945699237
We are now at 468
total reward 44.2087760033
We are now at 469
total reward 44.0157993526
saving models
average reward 109.540916474
We are now at 470
total reward 44.3659003797
We are now at 471
total reward 48.7072342255
We are now at 472
total reward 44.1225578245
We are now at 473
total reward 49.0290866504
We are now at 474
total reward 43.9325738138
saving models
average reward 109.544282177
We are now at 475
total reward 48.8953045469
We are now at 476
total reward 44.3097957278
We are now at 477
total reward 43.9814990278
We are now at 478
total reward 44.6909370678
We are now at 479
total reward 44.1689444034
saving models
average reward 109.588285938
We are now at 480
t